# Starter Notebook

Install and import required libraries

In [1]:
!pip install transformers datasets evaluate accelerate peft trl bitsandbytes
!pip install nvidia-ml-py3
! pip install optuna # U

In [2]:
! pip install hf_xet

In [3]:
import os
import pandas as pd
import torch
from transformers import RobertaModel, RobertaTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, RobertaForSequenceClassification, BitsAndBytesConfig, EarlyStoppingCallback # U
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, Dataset, ClassLabel
import pickle
from optuna.trial import Trial

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # loading the model weights in 4-bit precision
    bnb_4bit_use_double_quant= True, # using double quantization for 4-bit weights
    # nf4: non-uniform quantization scheme (a form of adaptive quantization) that tends to capture a wider range of weights more effectively!
    bnb_4bit_quant_type = "nf4", # using nf4 quantization type for 4-bit weights, another opt I know id fp4!
    bnb_4bit_compute_dtype= torch.bfloat16, # performing computations in bfloat16 format
    llm_int8_skip_modules= ["classifier"]
)

## Load Tokenizer and Preprocess Data

In [5]:
dataset = load_dataset('ag_news', split = 'train')
dataset[0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [6]:
base_model = 'roberta-base'

tokenizer = RobertaTokenizer.from_pretrained(base_model)

def preprocess(examples):
  # examples['text'] access the text content (text from the 'text' column) within current batch of examples
  # truncation = True: Ensures consistent input sizes
  # padding = True: pad tokens with special tokens to match the longest sequence in the batch
  tokenized = tokenizer(examples['text'], truncation = True, padding = True)
  return tokenized

tokenized_dataset = dataset.map(preprocess, batched = True, remove_columns = ['text']) # remove_columns = ['text']: this is a good practice as we would not need original 'text' column after tokenization!
tokenized_dataset = tokenized_dataset.rename_column('label', 'labels') # HuggingFace works well with 'labels' as a feature-name!

In [7]:
# print(dataset[0])
print(len(dataset))
print(len(tokenized_dataset))

120000
120000


In [8]:
# Extracting the number of classes and their names
num_labels = dataset.features['label'].num_classes # number of labels
class_names = dataset.features['label'].names # name of the corresponding classes
print(f"number of classes: {num_labels}")
print(f"label names: {class_names}")

# Creatig id2label mapping, needing for our classifier
id2label = {i: label for i, label in enumerate(class_names)}

# This pads each batch to the maximum length (within that batch) significantly reducing computational overhead and memory usage.
datacollator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

number of classes: 4
label names: ['World', 'Sports', 'Business', 'Sci/Tech']


## Load Pre-trained Model
Set up config for pretrained model and download it from hugging face

In [9]:
base = RobertaForSequenceClassification.from_pretrained(
    base_model, id2label = id2label
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Anything from here on can be modified

In [10]:
# Splitting the original training set
split_datasets = tokenized_dataset.train_test_split(test_size = 640, seed = 42, stratify_by_column = 'labels') # (U)
train_set = split_datasets['train']
train_set = train_set.shuffle(seed = 42) # U
test_set = split_datasets['test']
test_set = test_set.shuffle(seed = 42) # U

# print(len(train_set))
# print(len(test_set))

# print(train_set[0])

## Setup LoRA Config
Setup PEFT config and get peft model for finetuning

In [16]:
# Static code
# Peft Config (PEFT: Parameter Efficient Fine-Tuning!)
static_peft = LoraConfig(
    r = 10, # rank of updated matrices! (U)
    lora_alpha = 64, # scaling factor for learned updates controlling the strength of LORA adaption (U)
    lora_dropout = 0.1, # prob for the dropping LORA layers (helps in regularization)
    bias = 'none', # specifies if we want to apply bias to LORA updates
    target_modules = ['query', 'value'], # specifies which target modules do I need to apply LORA to, in our case (query and value modules of the pre-trained RobertaModel!)
    task_type = "SEQ_CLS", # determines what type of task are we performing and acc applies LORA!
)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
static_model = get_peft_model(base, static_peft).to(device)
static_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [19]:
# Determining the number of trainable parameters within the peft_model!
print("Trainable parameters:")
for name, param in static_model.named_parameters(): # named_parameters(): It is an iterator, a method in pytorch, yielding the name and parameter tensor for each parameter in the model
  if param.requires_grad: # requires_grad: boolean value (only the parameters that need to be updated during fine-tuning will have `requires_grad=True`.  All other parameters of the pre-trained model will be frozen (i.e., their values won't change).
    print(name)
    # print(len(param))

Trainable parameters:
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_A.default.weight
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_B.default.weight
base_model.model.roberta.encoder.layer.0.attention.self.value.lora_A.default.weight
base_model.model.roberta.encoder.layer.0.attention.self.value.lora_B.default.weight
base_model.model.roberta.encoder.layer.1.attention.self.query.lora_A.default.weight
base_model.model.roberta.encoder.layer.1.attention.self.query.lora_B.default.weight
base_model.model.roberta.encoder.layer.1.attention.self.value.lora_A.default.weight
base_model.model.roberta.encoder.layer.1.attention.self.value.lora_B.default.weight
base_model.model.roberta.encoder.layer.2.attention.self.query.lora_A.default.weight
base_model.model.roberta.encoder.layer.2.attention.self.query.lora_B.default.weight
base_model.model.roberta.encoder.layer.2.attention.self.value.lora_A.default.weight
base_model.model.roberta.encoder.layer.2.attention.sel

In [20]:
print('PEFT Model!')
static_model.print_trainable_parameters() # will print the total number of trainable parameters!

PEFT Model!
trainable params: 962,308 || all params: 125,611,016 || trainable%: 0.7661


## Training Setup

In [21]:
# This will track evaluation accuracy during training
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1) # will contain output probabilities of each class and argmax(-1) will help find out max val over each determining the class which it belongs to!
  accuracy = accuracy_score(labels, preds)
  precision = precision_score(labels, preds, average="weighted")
  recall = recall_score(labels, preds, average = "weighted")
  f1 = f1_score(labels, preds, average="weighted")
  return {
      'accuracy' : accuracy,
      'precision' : precision,
      'recall' : recall,
      'f1' : f1
  }

In [22]:
# Setup Training args
# training_args = TrainingArguments(
#     output_dir="hpo_results",
#     # eval_strategy="steps",
#     eval_steps=50,
#     # save_strategy="steps",
#     # save_steps = 50,
#     logging_steps=20,
#     max_steps=200,
#     warmup_ratio=0.1,
#     optim="adamw_torch",
#     fp16=False,
#     bf16=True,
#     per_device_eval_batch_size=64,
#     dataloader_num_workers=4,
#     dataloader_prefetch_factor=2,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_accuracy",
#     gradient_accumulation_steps = 2,
#     greater_is_better=True,
#     report_to=None,
# )

training_args = TrainingArguments(
    output_dir="hpo_results",
    max_steps=200,  # Shorter trials
    eval_steps=50,
    per_device_train_batch_size=32,  # Larger batches
    fp16=True,  # Use mixed precision
    dataloader_num_workers=4,
    report_to=None,
    metric_for_best_model="eval_accuracy",
    eval_strategy = "steps",
    greater_is_better=True
)


# Hyperparameter Fine-tuning using HyperOpt

In [23]:
from optuna.pruners import MedianPruner

In [24]:
# Initializing the model hyperparam-tuning fucntion
def model_init(trial: Trial = None):
      if trial is None:
        return static_model

      base = RobertaForSequenceClassification.from_pretrained(
          base_model, id2label = id2label
      )

      peft_config = LoraConfig(
        r = 10,
        lora_alpha = 64,
        lora_dropout = 0.1,
        bias = "none",
        target_modules= ["query", "value"],
        task_type = "SEQ_CLS"
      )
      peft_model = get_peft_model(base, peft_config).to(device)
      return peft_model

In [25]:
# Taking small subset of data for performing hyperparameter fine-tuning!
small_dataset = tokenized_dataset.train_test_split(train_size = 0.1, test_size = 0.05, seed = 42, stratify_by_column = "labels")
small_train_set = small_dataset["train"]
small_test_set = small_dataset["test"]

In [26]:
print(f"Small set training examples: {len(small_train_set)}")
print(f"Small set testing examples: {len(small_test_set)}")

Small set training examples: 12000
Small set testing examples: 6000


In [27]:
trainer = Trainer(
      # model = peft_model,
      model_init = model_init,
      args = training_args,
      compute_metrics = compute_metrics,
      train_dataset= small_train_set,
      eval_dataset= small_test_set,
      data_collator = datacollator,
      callbacks = [EarlyStoppingCallback(early_stopping_patience= 3)]
  )

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [28]:
best_run = trainer.hyperparameter_search(
    direction = "maximize", # maximize eval_acc
    backend = "optuna",
    n_trials = 20,
    pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=100),
    hp_space = lambda trial:{
        "learning_rate":               trial.suggest_loguniform("learning_rate", 1e-5, 5e-4),
        "per_device_train_batch_size": trial.suggest_categorical("batch_size", [8, 16, 32]),
        "gradient_accumulation_steps": trial.suggest_categorical("grad_accum", [1, 2, 4]),
        "weight_decay":                trial.suggest_categorical("weight_decay", [0.0, 0.01])
    },
)

print("Best hyperparameters:", best_run.hyperparameters)

[I 2025-04-18 22:37:55,752] A new study created in memory with name: no-name-a8248a7c-c896-4dc7-84e0-4f68e6be3f13
<ipython-input-28-b45aaca4fadf>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate":               trial.suggest_loguniform("learning_rate", 1e-5, 5e-4),
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current syste

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sk11634 (sk11634-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

eval/accuracy,▁██
eval/f1,▁██
eval/loss,█▂▁
eval/precision,▁▇█
eval/recall,▁██
eval/runtime,▅█▁
eval/samples_per_second,▄▁█
eval/steps_per_second,▄▁█
train/epoch,▁▅█
train/global_step,▁▅█
eval/accuracy,0.90683


Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,No log,0.295884,0.900667,0.901650,0.900667,0.900470
100,No log,0.266410,0.911000,0.910727,0.911000,0.910821
150,No log,0.259376,0.913000,0.914048,0.913000,0.912955
200,No log,0.251551,0.914833,0.914783,0.914833,0.914596


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: T

Best hyperparameters: {'learning_rate': 0.0004615402338932663, 'batch_size': 32, 'grad_accum': 2, 'weight_decay': 0.01}


In [31]:
best = best_run.hyperparameters
print(best)

# statically inserting for now (only tweaked num_epochs so!)
final_peft = LoraConfig(
    r = 10, # rank of updated matrices! (U)
    lora_alpha = 64, # scaling factor for learned updates controlling the strength of LORA adaption (U)
    lora_dropout = 0.1, # prob for the dropping LORA layers (helps in regularization)
    bias = "none", # specifies if we want to apply bias to LORA updates
    target_modules = ["query","value"],  # specifies which target modules do I need to apply LORA to, in our case (query and value modules of the pre-trained RobertaModel!)
    task_type = "SEQ_CLS", # determines what type of task are we performing and acc applies LORA!
)

# Reloading the base (the backbone never changes, we will fine‑tune on top of it)
final_base = RobertaForSequenceClassification.from_pretrained(
    # "roberta-base", id2label=id2label, quantization_config=quantization_config
    "roberta-base", id2label=id2label
)

final_model = get_peft_model(final_base, final_peft).to(device)

# Finally, rebuilding my TrainingArguments from the best values after hyperparameter fine-tuning!
output_dir = "result"
final_training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate = best["learning_rate"],
    per_device_train_batch_size = best["batch_size"],
    gradient_accumulation_steps = best["grad_accum"],
    weight_decay=best["weight_decay"],
    report_to=None,
    eval_strategy='steps',
    logging_steps=50,
    num_train_epochs=1,
    lr_scheduler_type='cosine_with_restarts',
    max_steps=2000,
    warmup_ratio=0.1,
    use_cpu=False,
    dataloader_num_workers=4,
    per_device_eval_batch_size=64,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True},
    eval_steps=400,
    fp16=True,
    optim = "adamw_torch",
    # bf16=True,
    save_strategy="steps",
    save_steps=400,
    save_total_limit=2,
    label_names=["labels"], # U
    load_best_model_at_end= True, # U
    metric_for_best_model= "eval_accuracy", # U
    greater_is_better= True, # U
)

final_trainer = Trainer(
    model = final_model,
    args = final_training_args,
    compute_metrics= compute_metrics,
    train_dataset= train_set,
    eval_dataset= test_set,
    data_collator = datacollator,
    callbacks = [EarlyStoppingCallback(early_stopping_patience= 3)]
)

{'learning_rate': 0.0004615402338932663, 'batch_size': 32, 'grad_accum': 2, 'weight_decay': 0.01}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Start Training

In [32]:
# # my actual trainer
# peft_lora_finetuning_trainer = (final_model)

# # for name, param in peft_lora_finetuning_trainer.named_parameters():
# #   if param.requires_grad:
# #     print(name)


# # my expected results once model is trained
# result = peft_lora_finetuning_trainer.train()
# print(result)

result = final_trainer.train()
print(result)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
400,0.259600,0.267410,0.921875,0.921510,0.921875,0.921524
800,0.216600,0.203550,0.923438,0.923392,0.923438,0.923402
1200,0.198000,0.202869,0.932813,0.932856,0.932813,0.932527
1600,0.175300,0.184811,0.931250,0.931057,0.931250,0.931051


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: T

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
400,0.259600,0.267410,0.921875,0.921510,0.921875,0.921524
800,0.216600,0.203550,0.923438,0.923392,0.923438,0.923402
1200,0.198000,0.202869,0.932813,0.932856,0.932813,0.932527
1600,0.175300,0.184811,0.931250,0.931057,0.931250,0.931051
2000,0.174200,0.183403,0.932813,0.932560,0.932813,0.932600


TrainOutput(global_step=2000, training_loss=0.2507305064201355, metrics={'train_runtime': 1618.8248, 'train_samples_per_second': 79.07, 'train_steps_per_second': 1.235, 'total_flos': 2.258345883631411e+16, 'train_loss': 0.2507305064201355, 'epoch': 1.0723860589812333})


## Evaluate Finetuned Model


### Performing Inference on Custom Input
Uncomment following functions for running inference on custom inputs

In [33]:
def classify(model, tokenizer, text):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  inputs = tokenizer(text, truncation = True, padding = True, return_tensors = "pt").to(device)
  output = model(**inputs)

  prediction = output.logits.argmax(dim = -1).item()

  print(f"\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}")
  return id2label[prediction]

In [34]:
classify(final_trainer.model, tokenizer, "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...")
classify(final_trainer.model, tokenizer, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.")


 Class: 1, Label: Sports, Text: Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...

 Class: 2, Label: Business, Text: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindlingand of ultra-cynics, are seeing green again.


'Business'

### Run Inference on eval_dataset

In [35]:
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

def evaluate_model(inference_model, dataset, labelled=True, batch_size=8, data_collator=None):
    """
    Evaluate a PEFT model on a dataset.

    Args:
        inference_model: The model to evaluate.
        dataset: The dataset (Hugging Face Dataset) to run inference on.
        labelled (bool): If True, the dataset includes labels and metrics will be computed.
                         If False, only predictions will be returned.
        batch_size (int): Batch size for inference.
        data_collator: Function to collate batches. If None, the default collate_fn is used.

    Returns:
        If labelled is True, returns a tuple (metrics, predictions)
        If labelled is False, returns the predictions.
    """
    # Create the DataLoader
    eval_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inference_model.to(device)
    inference_model.eval()

    all_predictions = []
    if labelled:
        metric = evaluate.load('accuracy')

    # Loop over the DataLoader
    for batch in tqdm(eval_dataloader):
        # Move each tensor in the batch to the device
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = inference_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        all_predictions.append(predictions.cpu())

        if labelled:
            # Expecting that labels are provided under the "labels" key.
            references = batch["labels"]
            metric.add_batch(
                predictions=predictions.cpu().numpy(),
                references=references.cpu().numpy()
            )

    # Concatenate predictions from all batches
    all_predictions = torch.cat(all_predictions, dim=0)

    if labelled:
        eval_metric = metric.compute()
        print("Evaluation Metric:", eval_metric)
        return eval_metric, all_predictions
    else:
        return all_predictions

In [36]:
# Check evaluation accuracy
_, _ = evaluate_model(final_trainer.model, test_set, True, 8, datacollator)

100%|██████████| 80/80 [00:03<00:00, 23.95it/s]

Evaluation Metric: {'accuracy': 0.9328125}


### Run Inference on unlabelled dataset

In [37]:
#Load your unlabelled data
unlabelled_dataset = pd.read_pickle("test_unlabelled.pkl")
test_dataset = unlabelled_dataset.map(preprocess, batched=True, remove_columns=["text"])
test_dataset = test_dataset.with_format("torch")  # Ensure no shuffle
unlabelled_dataset

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 8000
})

In [39]:
# Run inference and save predictions
preds = evaluate_model(final_trainer.model, test_dataset, False, 8, datacollator)
df_output = pd.DataFrame({
    'ID': range(len(preds)),
    'Label': preds.numpy()  # or preds.tolist()
})
df_output.to_csv(os.path.join(output_dir,"inference_output_trial9.csv"), index=False)
print("Inference complete. Predictions saved to inference_output_trial9.csv")

100%|██████████| 1000/1000 [00:41<00:00, 24.18it/s]

Inference complete. Predictions saved to inference_output_trial9.csv
